# Task I

You are given as input an unsorted array of n distinct numbers, where n is a power of 2. Give an algorithm that identifies the second-largest number in the array, and that uses at most $n + log_2 n - 2$ comparisons.

Solution algo from [this paper](http://users.csc.calpoly.edu/~dekhtyar/349-Spring2010/lectures/lec03.349.pdf)

<img src="img/second_max.jpeg" alt="Drawing" style="width: 400px;" align="left">

In [1]:
import math

def second_max(arr: list) -> int:
    
    max_chains = []
    for idx in range(len(arr) // 2):
        if arr[2 * idx] < arr[2 * idx + 1]:
            max_chains.append((arr[2 * idx + 1], [arr[2 * idx]]))
        else:
            max_chains.append((arr[2 * idx], [arr[2 * idx + 1]]))
            
    for power in range(2, round(math.log2(len(arr))) + 1):
        arr = max_chains
        max_chains = []
        for idx in range(len(arr) // 2):
            if arr[2 * idx][0] < arr[2 * idx + 1][0]:
                max_chains.append((
                    arr[2 * idx + 1][0], 
                    arr[2 * idx + 1][1] + [arr[2 * idx][0]]))
            else:
                max_chains.append((
                    arr[2 * idx][0], 
                    arr[2 * idx][1] + [arr[2 * idx + 1][0]]))
    return max(max_chains[0][1])
    

In [2]:
from helpers_functions import test_function

In [3]:
import random

test_cases_sm = [
    ([arr], list(sorted(arr))[-2]) for arr in [
        list(range(2 ** i)) for i in range(1, 10)
    ]
]

test_function(second_max, test_cases_sm)

Test 0 passed
Test 1 passed
Test 2 passed
Test 3 passed
Test 4 passed
Test 5 passed
Test 6 passed
Test 7 passed
Test 8 passed


# Task II

You are a given a unimodal array of n distinct elements, meaning that its entries are in increasing order up until its maximum element, after which its elements are in decreasing order. Give an algorithm to compute the maximum element that runs in $O(\log n)$ time.

In [4]:
# The idea is to watch on the nearest numbers to the arr_m, where m = len(arr) / 2
# lets consider smallest cases, to understand correctly conditions inside function:
# [0, 1, 0, -1, -2] m=5//2=2 down down +
#        ^
# [0, 1, 2, 3, 2] m=5//2=2 up up -
#        ^
# [0, 1, 2, 1, 0] m=5//2=2 up down -
#        ^
# [0, 1, 0] m=3//2=1 up down -
#     ^
# [0, 1] m=2//2=1 up -
#     ^
# [1, 0] m=2//2=1 down +
#     ^

In [5]:
def unisearch(arr: list) -> int:
    if len(arr) <= 1:
        return arr[0] if len(arr) == 1 else None
    m = len(arr) // 2
    if arr[m] > arr[m - 1]: # up
        return unisearch(arr[m:])
    else: # down
        return unisearch(arr[:m])

In [6]:
test_cases_us = [
    ([arr], max(arr)) for arr in [
        [0, 1, 0, -1, -2],
        [0, 1, 2, 3, 2],
        [0, 1, 2, 1, 0],
        [0, 1, 0],
        [0, 1],
        [1, 0],
        [1],
        [10, 9, 8, 7],
        [7, 8, 9, 10]
    ]
]

test_function(unisearch, test_cases_us)

Test 0 passed
Test 1 passed
Test 2 passed
Test 3 passed
Test 4 passed
Test 5 passed
Test 6 passed
Test 7 passed
Test 8 passed


how we see, solving smallest cases is very usefull - I coded correct solution from the first try, which is very rare for me, if I just start to solve problem without pre-worl

In [7]:
from helpers_functions import time_inp_data

In [31]:
test_time_cases_us = [
    [list(range(1, 2 ** i + 1)) + list(range(2 ** i + 2, 1))[::-1]] for i in range(15) 
]

time_diff = time_inp_data(unisearch, test_time_cases_us, 1000)

In [32]:
time_diff

array([2.24337349, 1.56552095, 2.14236707, 0.85780624, 1.23688632,
       1.24947178, 1.1869791 , 1.41416506, 1.37741959, 1.37258385,
       1.49996194, 1.63803603, 1.73373093, 1.77464462])

$\frac{\log 2N}{\log N}=1 + \log_N{2} \rightarrow 1$ 

now we vividly see, that time tests contradict with theoretical time approximation!

lets count steps:

In [25]:
from typing import Tuple

def unisearch_steps(arr: list, steps: int=0) -> Tuple[int, int]:
    if len(arr) <= 1:
        return arr[0], steps + 1
    m = len(arr) // 2
    if arr[m] > arr[m - 1]: # up
        return unisearch_steps(arr[m:], steps + 1)
    else: # down
        return unisearch_steps(arr[:m], steps + 1)
    
unisearch_steps(test_time[-1][0]), unisearch_steps(test_time[5][0])

((16384, 15), (32, 6))

well, we do as much recursive calls as needed, so probably the problem is in recursion tree realization itself.

Let's rewrite the function without recursion

Again it is impossible to write correct algorithm without considering smallest cases.

In [29]:
# [0, 1]
# l = 0 -> 1=m
# r = 2
# m = 1
# correct answer l

# [1, 0]
# l = 0
# r = 2->1=m
# m = 1
# again correct answer l, we are lucky;)

In [27]:
def unisearch_no_recursion(arr: list) -> int:
    l = 0
    r = len(arr)
    while l < r - 1:
        m = (l + r) // 2
        if arr[m] > arr[m - 1]: # up
            l = m
        else: # down
            r = m
    return arr[l]

In [28]:
test_function(unisearch_no_recursion, test_cases_us)

Test 0 passed
Test 1 passed
Test 2 passed
Test 3 passed
Test 4 passed
Test 5 passed
Test 6 passed
Test 7 passed
Test 8 passed


Great, considering test cases helped to write correct algo from the first try!

In [33]:
time_diff_no_rec = time_inp_data(unisearch_no_recursion, test_time_cases_us, 10000)
time_diff_no_rec

array([1.72964535, 1.26727312, 1.37902353, 1.1084855 , 1.13081395,
       1.09356667, 1.14309394, 1.1837644 , 1.4424152 , 1.1231413 ,
       0.9284418 , 1.02962996, 1.0743665 , 1.07148116])

this seems to be more near to the truth, but again not ideal converge

# Task III

In [11]:
import numpy as np
np.ndarray

numpy.ndarray